## BIS CPMI Cashless Payments Forecasting using Hierarchical Time Series (HTS)

### Overview
This notebook demonstrates a production-grade payment forecasting system using **Hierarchical Time Series (HTS)** modelling to predict card and e-money transaction volumes across global payment channels.

### Goal
Forecast cashless payment volumes across a 4-level hierarchy and identify **high-risk channels** that are projected to underperform the market. We then enable targeted payment optimisation strategies.

### Hierarchy Structure
The model forecasts volumes at every level of this hierarchy:
- **Total** (global aggregate)
  - **Region** (mapped from reporting countries: Europe, North America, APAC, etc.)
    - **Method** (payment instrument type: Card/e-Money, Credit Transfers, etc.)
      - **Channel** (terminal location and device technology: Contactless, Magstripe, etc.)

### Data Source
Using the **Bank for International Settlements (BIS) Committee on Payments and Market Infrastructures (CPMI)** public dataset on cashless payment statistics, which is a comprehensive source covering 40+ countries and multiple payment instrument types.

### Methodology
1. **Data Ingestion**: Filter BIS CPMI data to annual transaction counts for card/e-money payments
2. **Schema Mapping**: Transform BIS dimensions → (Region, Method, Channel) hierarchy
3. **Aggregation**: Build the full hierarchical structure with summing matrix
4. **Base Forecasting**: Generate independent forecasts for each node using Facebook Prophet
5. **Hierarchical Reconciliation**: Apply MinTrace reconciliation to ensure forecasts are coherent (sub-levels sum to parents)
6. **Risk Segmentation**: Calculate growth gaps and market share shifts to identify optimisation targets

### Key Outputs
- **Reconciled forecasts** for all hierarchy nodes (coherent aggregation guaranteed)
- **Top 15 optimisation targets** ranked by growth underperformance and share loss
- **Risk-segmented visualisation** showing channels threatened by volume contraction, competitive erosion, or legacy tech sunset

---
## Dependencies

This pipeline uses:
- **pandas/numpy**: Data manipulation and numerical operations
- **Prophet**: Facebook's time series forecasting library (automatic trend + seasonality detection)
- **hierarchicalforecast**: Suite for hierarchical reconciliation (MinTrace algorithm)
- **plotly**: Interactive visualisations for stakeholder reporting

In [ ]:
from __future__ import annotations

import re

import pandas as pd
import numpy as np

from prophet import Prophet

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import MinTrace
from hierarchicalforecast.utils import aggregate

import logging # Suppress cmdstanpy INFO log messages

import plotly.express as px

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)

c:\Users\beckk\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load BIS CPMI Cashless Payments data
csv_path = r"<enter file path>"
df = pd.read_csv(csv_path)
print('Rows, Columns:', df.shape)
display(df.head(2))

Rows, Columns: (17956, 31)


,STRUCTURE,STRUCTURE_ID,ACTION,FREQ:Frequency,REP_CTY:Reporting country,MEASURE:Measure,INSTRUMENT_TYPE:Instrument type,DIRECTION:Direction,TERMINAL_LOC:Terminal location,CARD_ISS_LOC:Cards issuance location,DEV_STATE_TECH:Device state / technology,TERMINAL_TYPE:Terminal type,CARD_FCT:Card function,PAYMT_SPEED:Payment speed,ISSUER_OR_ONUS:Issuer or on-us payments,TIME_PERIOD:Time period or range,OBS_VALUE:Observation Value,UNIT_MULT:Unit Multiplier,UNIT_MEASURE:Unit of measure,TITLE:Title,TIME_FORMAT:Time Format,TABLE:Table,DECIMALS:Decimals,COMMENT_TS:Time series comment,COLLECTION:Collection Indicator,AVAILABILITY:Availability,OLD_TABLE:Old table,OBS_STATUS:Observation Status,OBS_PRE_BREAK:Pre-Break Observation,OBS_CONF:Observation confidentiality,COMMENT_OBS:Observation comment
0,dataflow,BIS:WS_CPMI_CASHLESS(1.0): CPMI cashless payme...,I,A: Annual,SE: Sweden,N: Number,B: Credit transfers,A: All,Z: Not applicable,Z: Not applicable,"Q: Device-present, paper based",A: All,Z: Not applicable,A: All,A: All issuers and cashless payments,2012,70.0,6: Millions,373: Pure Number,"Sweden - Volume of cashless payments, paper ba...",NaN,5,3: Three,NaN,S: Summed through period,A: All users,7.0,A: Normal value,NaN,F: Free,NaN
1,dataflow,BIS:WS_CPMI_CASHLESS(1.0): CPMI cashless payme...,I,A: Annual,SE: Sweden,N: Number,B: Credit transfers,A: All,Z: Not applicable,Z: Not applicable,"Q: Device-present, paper based",A: All,Z: Not applicable,A: All,A: All issuers and cashless payments,2013,67.0,6: Millions,373: Pure Number,"Sweden - Volume of cashless payments, paper ba...",NaN,5,3: Three,NaN,S: Summed through period,A: All users,7.0,A: Normal value,NaN,F: Free,NaN


---
## Data Loading

Loading the BIS CPMI dataset, which contains historical payment statistics across multiple dimensions:
- Countries (40+ reporting jurisdictions)
- Instrument types (cards, credit transfers, direct debits, etc.)
- Terminal locations (domestic vs. cross-border)
- Device technologies (contactless, magstripe, etc.)
- Time periods (annual, quarterly, monthly data available)

---
## Step 1: Data Transformation & Hierarchy Mapping

### What This Section Does:
1. **Filters** BIS data to annual transaction counts for card/e-money instruments (prefix "F:")
2. **Maps** BIS dimensions to a clean 3-level hierarchy:
   - Country → Region (using `COUNTRY_TO_REGION` mapping)
   - Instrument Type → Method (Card_eMoney, CreditTransfers, etc.)
   - Terminal Location + Device Technology → Channel
3. **Handles outliers**: Excludes 2020 data (COVID-19 anomaly) from forecasting
4. **Normalises values**: Applies BIS unit multipliers to get actual transaction counts

### Key Functions:
- `strip_code_prefix()`: Cleans BIS code prefixes (e.g., "GB: United Kingdom" → "United Kingdom")
- `parse_time_period_to_ds()`: Converts various BIS date formats to pandas Timestamps
- `map_method()`: Translates BIS instrument codes to business-friendly payment methods
- `map_channel()`: Combines terminal location and device technology into channel labels
- `build_leaf_timeseries()`: Main ETL pipeline that produces the leaf-level time series

In [3]:
# Configuration (simple variables)
FREQ = 'A: Annual'
MEASURE = 'N: Number'
UNIT_MEASURE = '373: Pure Number'
INSTRUMENT_PREFIXES = ('F:',)
HORIZON = 3
MAX_SERIES = None

# Prophet settings
GROWTH = 'linear'  # try 'flat' if trend looks too aggressive
EXCLUDE_YEARS = [2020]  # remove outlier years such as the COVID dip; set [] to keep all years

# Minimal country->region mapping (extend as needed)
COUNTRY_TO_REGION = {
    # Europe
    'United Kingdom': 'Europe',
    'Germany': 'Europe',
    'France': 'Europe',
    'Italy': 'Europe',
    'Spain': 'Europe',
    'Netherlands': 'Europe',
    'Sweden': 'Europe',
    'Switzerland': 'Europe',
    'Norway': 'Europe',
    'Denmark': 'Europe',
    'Poland': 'Europe',
    'Ireland': 'Europe',
    'Belgium': 'Europe',
    'Portugal': 'Europe',
    'Greece': 'Europe',
    'Czechia': 'Europe',
    'Hungary': 'Europe',
    'Austria': 'Europe',
    # North America
    'United States': 'North America',
    'Canada': 'North America',
    'Mexico': 'North America',
    # APAC
    'Japan': 'APAC',
    'Australia': 'APAC',
    'New Zealand': 'APAC',
    'China': 'APAC',
    'Hong Kong SAR': 'APAC',
    'Singapore': 'APAC',
    'Korea': 'APAC',
    'India': 'APAC',
    'Indonesia': 'APAC',
    'Türkiye': 'EMEA',
    # LatAm
    'Brazil': 'LatAm',
    'Argentina': 'LatAm',
    'Chile': 'LatAm',
    'Colombia': 'LatAm',
    # MEA
    'South Africa': 'MEA',
    'Saudi Arabia': 'MEA',
    'United Arab Emirates': 'MEA',
    'Israel': 'MEA',
    'Russia': 'EMEA',
}

TERM_LOC_LABEL = {
    'I: Inside the country': 'Domestic',
    'O: Outside the country': 'CrossBorder',
    'A: All': 'AllLoc',
}

DEV_STATE_LABEL = {
    'S: Device-present, contactless': 'Contactless',
    'P: Device-present, all': 'CardPresent',
    'T: Device-present, magstripe': 'Magstripe',
    'A: All': 'AllPresent',
}


def strip_code_prefix(value: str) -> str:
    """Turn e.g. 'GB: United Kingdom' -> 'United Kingdom'."""
    s = str(value)
    return re.sub(r'^[^:]+:\s*', '', s).strip()


def parse_time_period_to_ds(time_period: str) -> pd.Timestamp:
    """Parse BIS TIME_PERIOD strings to a pandas Timestamp."""
    s = str(time_period)
    # Annual: YYYY
    m = re.fullmatch(r'(\d{4})', s)
    if m:
        return pd.Timestamp(int(m.group(1)), 1, 1)

    # Quarterly: YYYY-Q[1-4]
    m = re.fullmatch(r'(\d{4})-Q([1-4])', s)
    if m:
        year = int(m.group(1))
        q = int(m.group(2))
        month = 1 + (q - 1) * 3
        return pd.Timestamp(year, month, 1)

    # Monthly: YYYY-MM
    m = re.fullmatch(r'(\d{4})-(\d{2})', s)
    if m:
        return pd.Timestamp(int(m.group(1)), int(m.group(2)), 1)

    # Fallback
    return pd.to_datetime(s, errors='coerce')


def map_method(instrument_type: str) -> str:
    s = str(instrument_type)
    if s.startswith('F:'):
        return 'Card_eMoney'
    if s.startswith('B:'):
        return 'CreditTransfers'
    if s.startswith('C:'):
        return 'DirectDebits'
    if s.startswith('D:'):
        return 'Cheques'
    if s.startswith('A:'):
        return 'CashlessAll'
    if s.startswith('M:'):
        return 'CashWithdrawals'
    return 'Other'


def map_channel(row: pd.Series) -> str:
    terminal_loc_full = str(row.get('TERMINAL_LOC:Terminal location', 'A: All'))
    dev_state_full = str(row.get('DEV_STATE_TECH:Device state / technology', 'A: All'))

    terminal_loc = TERM_LOC_LABEL.get(terminal_loc_full, strip_code_prefix(terminal_loc_full) or 'AllLoc')
    dev_state = DEV_STATE_LABEL.get(dev_state_full, strip_code_prefix(dev_state_full) or 'AllPresent')

    return f"{terminal_loc}__{dev_state}"


def build_leaf_timeseries(df_raw: pd.DataFrame) -> pd.DataFrame:
    df0 = df_raw.copy()

    df0 = df0[df0['FREQ:Frequency'].astype(str) == FREQ].copy()
    df0 = df0[df0['MEASURE:Measure'].astype(str) == MEASURE].copy()
    df0 = df0[df0['UNIT_MEASURE:Unit of measure'].astype(str) == UNIT_MEASURE].copy()

    df0 = df0[
        df0['INSTRUMENT_TYPE:Instrument type']
        .astype(str)
        .apply(lambda s: any(s.startswith(p) for p in INSTRUMENT_PREFIXES))
    ].copy()

    # Parse time
    df0['ds'] = df0['TIME_PERIOD:Time period or range'].map(parse_time_period_to_ds)
    df0 = df0[df0['ds'].notna()].copy()

    # Apply unit multiplier if present (BIS typically: y * 10**UNIT_MULT)
    df0['OBS_VALUE:Observation Value'] = pd.to_numeric(df0['OBS_VALUE:Observation Value'], errors='coerce')
    df0['UNIT_MULT:Unit Multiplier'] = pd.to_numeric(df0['UNIT_MULT:Unit Multiplier'], errors='coerce').fillna(0)
    df0['y'] = df0['OBS_VALUE:Observation Value'] * (10 ** df0['UNIT_MULT:Unit Multiplier'])

    # Hierarchy dims
    df0['Country'] = df0['REP_CTY:Reporting country'].map(strip_code_prefix)
    df0['Region'] = df0['Country'].map(COUNTRY_TO_REGION).fillna('Other')
    df0['Method'] = df0['INSTRUMENT_TYPE:Instrument type'].map(map_method)
    df0['Channel'] = df0.apply(map_channel, axis=1)

    leaf = (
        df0.groupby(['ds', 'Region', 'Method', 'Channel'], as_index=False)['y']
        .sum()
        .sort_values(['Region', 'Method', 'Channel', 'ds'])
    )

    leaf = leaf.replace([np.inf, -np.inf], np.nan).dropna(subset=['y'])
    leaf = leaf[leaf['y'] >= 0].copy()

    return leaf

leaf_df = build_leaf_timeseries(df)
print('Leaf rows:', leaf_df.shape)
print('Leaf coverage (unique):', {
    'Region': leaf_df['Region'].nunique(),
    'Method': leaf_df['Method'].nunique(),
    'Channel': leaf_df['Channel'].nunique(),
    'ds': leaf_df['ds'].nunique(),
})

display(leaf_df.head(5))

Leaf rows: (456, 5)
Leaf coverage (unique): {'Region': 6, 'Method': 1, 'Channel': 9, 'ds': 12}


,ds,Region,Method,Channel,y
0,2012-01-01,APAC,Card_eMoney,AllLoc__AllPresent,67562.206
38,2013-01-01,APAC,Card_eMoney,AllLoc__AllPresent,61516.037
76,2014-01-01,APAC,Card_eMoney,AllLoc__AllPresent,75003.931
114,2015-01-01,APAC,Card_eMoney,AllLoc__AllPresent,93269.131
152,2016-01-01,APAC,Card_eMoney,AllLoc__AllPresent,114930.560


---
## Step 2: Build the Hierarchical Structure

### What This Section Does:
Creates the full hierarchy by aggregating leaf-level series upward through all levels:
- **Level 4 (Leaves)**: Total/Region/Method/Channel (e.g., "Total/Europe/Card_eMoney/Domestic__Contactless")
- **Level 3**: Total/Region/Method (e.g., "Total/Europe/Card_eMoney")
- **Level 2**: Total/Region (e.g., "Total/Europe")
- **Level 1 (Root)**: Total

The `aggregate()` function from hierarchicalforecast:
- Generates aggregated time series for all parent nodes
- Creates the **summing matrix (S)** that defines parent-child relationships
- Returns **tags** dictionary for easy lookup of nodes at each level

This structure enables coherent forecasting where all sub-levels mathematically sum to their parents.

In [4]:
# Leaf time series into hierarchicalforecast format
leaf_ts = leaf_df.copy()
leaf_ts['Total'] = 'Total'
leaf_ts['unique_id'] = (
    leaf_ts['Total'].astype(str) + '/' +
    leaf_ts['Region'].astype(str) + '/' +
    leaf_ts['Method'].astype(str) + '/' +
    leaf_ts['Channel'].astype(str)
)

# Hierarchy specification for aggregation
# We add an explicit 'Total' column because aggregate() in this version can't handle an empty level.
spec = [
    ['Total'],
    ['Total', 'Region'],
    ['Total', 'Region', 'Method'],
    ['Total', 'Region', 'Method', 'Channel'],
]

Y_agg_df, S, tags = aggregate(leaf_ts[['ds','y','Total','Region','Method','Channel']], spec=spec)

all_series = sorted(Y_agg_df['unique_id'].unique().tolist())
print('Total series count (all nodes):', len(all_series))
print('Has Total:', 'Total' in all_series)

display(Y_agg_df.head(5))

Total series count (all nodes): 51
Has Total: True


,unique_id,ds,y
0,Total,2012-01-01,8.124837e+05
1,Total,2013-01-01,8.879425e+05
2,Total,2014-01-01,1.014174e+06
3,Total,2015-01-01,1.128042e+06
4,Total,2016-01-01,1.256387e+06


---
## Step 3: Generate Base Forecasts with Prophet

### Why Prophet?
Prophet is ideal for payment data because it:
- Automatically detects trend changes (crucial for understanding payment method shifts)
- Handles missing data and outliers gracefully
- Requires minimal hyperparameter tuning
- Produces interpretable forecasts with uncertainty intervals

### What This Section Does:
1. Loops through **every node** in the hierarchy (Total, all Regions, all Methods, all Channels)
2. For each series:
   - Excludes outlier years (2020 COVID impact)
   - Fits a Prophet model with linear growth
   - Generates a 3-year forecast (`HORIZON = 3`)
   - Clips predictions to non-negative values (transaction counts can't be negative)
3. Combines all forecasts into `Y_hat_df` (base forecasts before reconciliation)

**Note**: These base forecasts are incoherent—child nodes don't necessarily sum to parents. Step 4 fixes this.

In [9]:
all_series = sorted(Y_agg_df['unique_id'].unique().tolist())
print('Total series count (all nodes):', len(all_series))

# Determine annual frequency string for Prophet future dataframe
# BIS annual is represented as Jan-01 each year -> use 'YS' (year start)
PROP_FREQ = 'YS'

logging.getLogger("cmdstanpy").setLevel(logging.WARNING)

def prophet_base_forecast(series_df: pd.DataFrame, horizon: int, freq: str) -> pd.DataFrame:
    series_df = series_df.sort_values('ds')
    series_df = series_df[['ds', 'y']].dropna()

    # Optional outlier handling (e.g. COVID dip)
    if EXCLUDE_YEARS:
        series_df = series_df[~series_df['ds'].dt.year.isin(EXCLUDE_YEARS)].copy()

    # Prophet needs at least 2 points; realistically use >= 4 for stability
    if len(series_df) < 4:
        return pd.DataFrame(columns=['ds', 'yhat'])

    m = Prophet(
        growth=GROWTH,
        yearly_seasonality='auto',
        weekly_seasonality=False,
        daily_seasonality=False,
    )
    m.fit(series_df)

    # Ensure the forecast starts immediately after the last historical period
    future = m.make_future_dataframe(periods=horizon, freq=freq, include_history=False)
    fcst = m.predict(future)

    out = fcst[['ds', 'yhat']].copy()
    out['yhat'] = out['yhat'].clip(lower=0)  # volumes can't be negative
    return out

base_forecasts = []
unique_ids = all_series
if MAX_SERIES is not None:
    unique_ids = unique_ids[: MAX_SERIES]

for uid in unique_ids:
    hist = Y_agg_df[Y_agg_df['unique_id'] == uid]
    fc = prophet_base_forecast(hist, horizon=HORIZON, freq=PROP_FREQ)
    if fc.empty:
        continue
    fc.insert(0, 'unique_id', uid)
    base_forecasts.append(fc)

Y_hat_df = pd.concat(base_forecasts, ignore_index=True) if base_forecasts else pd.DataFrame(columns=['unique_id', 'ds', 'yhat'])
print('Base forecast rows:', Y_hat_df.shape)

display(Y_hat_df.head(5))

Total series count (all nodes): 51
Base forecast rows: (153, 3)


,unique_id,ds,yhat
0,Total,2024-01-01,2.821145e+06
1,Total,2025-01-01,2.926090e+06
2,Total,2026-01-01,3.179304e+06
3,Total/APAC,2024-01-01,7.323811e+05
4,Total/APAC,2025-01-01,7.645243e+05


---
## Step 4: Hierarchical Reconciliation

### The Problem:
Base forecasts from Prophet are incoherent—if you sum up all channel forecasts, they won't match the Total forecast. This creates confusion and erodes stakeholder trust.

### The Solution: MinTrace Reconciliation
The **MinTrace algorithm** adjusts base forecasts to ensure perfect mathematical coherence:
- Uses the summing matrix **S** to understand parent-child relationships
- Applies Ordinary Least Squares (OLS) to minimise forecast error variance
- Produces **reconciled forecasts** where all levels sum correctly

### What This Section Does:
1. Applies MinTrace reconciliation to `Y_hat_df` (base forecasts)
2. Validates coherence: checks that sum of leaf forecasts = Total forecast
3. Outputs `Y_rec_df` with reconciled column `yhat/MinTrace_method-ols`

**Key Benefit**: Executives can drill down from Total → Region → Method → Channel and the numbers will always add up perfectly.

In [6]:
reconcilers = [MinTrace(method='ols')]
reconciler = HierarchicalReconciliation(reconcilers=reconcilers)

# Reconcile (returns base 'yhat' and reconciled 'yhat/<method>')
Y_rec_df = reconciler.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_agg_df, S_df=S, tags=tags)

print('Reconciled forecast rows:', Y_rec_df.shape)
print('Columns:', Y_rec_df.columns.tolist())

# Pick the reconciled column
model_cols = [c for c in Y_rec_df.columns if c not in ['unique_id', 'ds']]
rec_cols = [c for c in model_cols if c.startswith('yhat/')]
if not rec_cols:
    raise ValueError(f"No reconciled yhat column found; got {model_cols}")
REC_COL = rec_cols[0]
print('Using reconciled column:', REC_COL)

# Quick reconciliation check at forecast horizon: do leaf sums match Total?
leaf_ids = list(tags['Total/Region/Method/Channel'])
root_id = 'Total'
last_fcst_ds = Y_rec_df['ds'].max()

leaf_sum = Y_rec_df[(Y_rec_df['unique_id'].isin(leaf_ids)) & (Y_rec_df['ds'] == last_fcst_ds)][REC_COL].sum()
root_val = Y_rec_df[(Y_rec_df['unique_id'] == root_id) & (Y_rec_df['ds'] == last_fcst_ds)][REC_COL].iloc[0]
print(f"\nReconciliation check at {last_fcst_ds.date()}: leaf sum={leaf_sum:,.3f} vs Total={root_val:,.3f} (diff={leaf_sum-root_val:,.6f})")

Reconciled forecast rows: (153, 4)
Columns: ['unique_id', 'ds', 'yhat', 'yhat/MinTrace_method-ols']
Using reconciled column: yhat/MinTrace_method-ols

Reconciliation check at 2026-01-01: leaf sum=3,166,486.231 vs Total=3,166,486.231 (diff=0.000000)


---
## Forecast Visualisation

### What These Charts Show:
Using **reconciled forecasts only** (ensuring all levels sum correctly):

**Chart 1: Total Market Trajectory**
- Global card/e-money transaction volumes (historical + 3-year forecast)
- Shows overall market growth trend

**Chart 2: Regional Performance**
- Forecast trajectories for each geographic region
- Useful for identifying regional winners and losers
- Line style distinguishes history (solid) from forecast (dashed)

**Chart 3: Regional Snapshot Bar**
- Final forecast year by region
- Quick comparison of relative regional volumes


In [7]:
# Build chart dataframe: history + reconciled forecasts for Total and Regions
history_df = Y_agg_df.copy()
forecast_df = Y_rec_df[['unique_id', 'ds', REC_COL]].rename(columns={REC_COL: 'y'})

history_df['type'] = 'History'
forecast_df['type'] = 'Forecast'

# Stitch: add the last historical point to the forecast trace so the line is continuous
last_hist = (
    history_df.sort_values('ds')
    .groupby('unique_id', as_index=False)
    .tail(1)
    .assign(type='Forecast')
)
forecast_df = pd.concat([last_hist[['unique_id', 'ds', 'y', 'type']], forecast_df], ignore_index=True)

chart_df = pd.concat([
    history_df[['unique_id', 'ds', 'y', 'type']],
    forecast_df[['unique_id', 'ds', 'y', 'type']],
], ignore_index=True)

# 1) Total only
chart_total = chart_df[chart_df['unique_id'] == 'Total']
fig_total = px.line(
    chart_total,
    x='ds',
    y='y',
    color='type',
    title='Total authorisation volumes (history + forecast)',
)
fig_total.update_layout(yaxis_title='Transactions (count)', legend_title_text='')
fig_total.show()

# 2) Regions (history + forecast)
region_ids = list(tags['Total/Region'])
chart_regions = chart_df[chart_df['unique_id'].isin(region_ids)]
fig_regions = px.line(
    chart_regions,
    x='ds',
    y='y',
    color='unique_id',
    line_dash='type',
    title='Regional authorisation volumes (history + forecast)',
)
fig_regions.update_layout(yaxis_title='Transactions (count)', legend_title_text='Region / Type')
fig_regions.show()

# 3) Forecast snapshot: last horizon year by region
last_fcst_ds = forecast_df['ds'].max()
fcst_region = (
    forecast_df[forecast_df['unique_id'].isin(region_ids) & (forecast_df['ds'] == last_fcst_ds)]
    .sort_values('y', ascending=False)
)
fig_region_bar = px.bar(
    fcst_region,
    x='unique_id',
    y='y',
    title=f'Regional forecast at {last_fcst_ds.date()}',
)
fig_region_bar.update_layout(xaxis_title='Region', yaxis_title='Transactions (count)')
fig_region_bar.show()

---
## Payment Optimisation Targets

### Business Context:
Not all payment channels will grow equally. Some will underperform the market due to:
- Declining customer preference (e.g., magstripe cards)
- Competitive displacement (e.g., contactless replacing chip)
- Regulatory headwinds or security concerns

This section identifies which channels need intervention to avoid market share loss.

### Methodology:
For each leaf channel, we calculate:

**1. Growth Metrics:**
- `cagr`: Channel's compound annual growth rate
- `growth_gap_pp`: How many percentage points the channel lags behind Total CAGR

**2. Market Share Metrics:**
- `share_delta_bp`: Market share change in basis points (100 bps = 1%)
- Negative values = losing share to other channels

**3. Risk Factor Classification:**
Each channel is tagged with a risk driver:
- **Volume Contraction**: Channel is physically shrinking (CAGR < 0%)
- **Significant Growth Lag**: Growing, but >5pp slower than the market
- **Competitive Erosion**: Losing >20 basis points of market share
- **Legacy Tech Sunset**: Magstripe channels (known obsolescence)

**4. Optimisation Score:**
Combines growth gap and share loss into a single priority metric, a higher score = higher urgency for intervention.

### Outputs:
- **Table**: Top 15 channels ranked by optimisation score
- **Horizontal Bar Chart**: Top 10 declining channels, colour-coded by risk factor, with hover details for stakeholder review

In [11]:
# Build optimisation targets at leaf level using reconciled forecasts
leaf_ids = list(tags['Total/Region/Method/Channel'])
root_id = 'Total'

hist_last_ds = Y_agg_df['ds'].max()
fcst_last_ds = Y_rec_df['ds'].max()

# Total trend
root_hist = Y_agg_df[(Y_agg_df['unique_id'] == root_id) & (Y_agg_df['ds'] == hist_last_ds)]['y'].iloc[0]
root_fcst = Y_rec_df[(Y_rec_df['unique_id'] == root_id) & (Y_rec_df['ds'] == fcst_last_ds)][REC_COL].iloc[0]

h = HORIZON if HORIZON > 0 else 1
root_cagr = (root_fcst / root_hist) ** (1 / h) - 1

# Leaf metrics
leaf_hist = Y_agg_df[(Y_agg_df['unique_id'].isin(leaf_ids)) & (Y_agg_df['ds'] == hist_last_ds)][['unique_id', 'y']]
leaf_fcst = Y_rec_df[(Y_rec_df['unique_id'].isin(leaf_ids)) & (Y_rec_df['ds'] == fcst_last_ds)][['unique_id', REC_COL]]
leaf_fcst = leaf_fcst.rename(columns={REC_COL: 'y_fcst'})

leaf_metrics = leaf_hist.merge(leaf_fcst, on='unique_id', how='inner')
leaf_metrics['cagr'] = (leaf_metrics['y_fcst'] / leaf_metrics['y']) ** (1 / h) - 1
leaf_metrics['growth_gap_vs_total'] = root_cagr - leaf_metrics['cagr']

# Share shift
leaf_metrics['share_hist'] = leaf_metrics['y'] / root_hist
leaf_metrics['share_fcst'] = leaf_metrics['y_fcst'] / root_fcst
leaf_metrics['share_delta'] = leaf_metrics['share_fcst'] - leaf_metrics['share_hist']

# Split hierarchy into columns
parts = leaf_metrics['unique_id'].str.split('/', expand=True)
leaf_metrics['Region'] = parts[1]
leaf_metrics['Method'] = parts[2]
leaf_metrics['Channel'] = parts[3]

# Derived metrics for reporting/logic
leaf_metrics['cagr_pct'] = (leaf_metrics['cagr'] * 100).round(2)
leaf_metrics['growth_gap_pp'] = (leaf_metrics['growth_gap_vs_total'] * 100).round(2)
leaf_metrics['share_delta_bp'] = (leaf_metrics['share_delta'] * 10000).round(1)

# Risk factor classification
risk_conditions = [
    leaf_metrics['cagr_pct'] < 0,
    leaf_metrics['growth_gap_pp'] > 5,
    leaf_metrics['share_delta_bp'] < -20,
 ]
risk_choices = [
    'Volume Contraction',
    'Significant Growth Lag',
    'Competitive Erosion',
 ]
leaf_metrics['risk_factor'] = np.select(risk_conditions, risk_choices, default='')
leaf_metrics.loc[leaf_metrics['Channel'].str.contains('Magstripe', case=False, na=False), 'risk_factor'] = 'Legacy Tech Sunset'

# Score: prioritise growth underperformance and share loss
leaf_metrics['optimisation_score'] = (leaf_metrics['growth_gap_vs_total'] * 100) + (-leaf_metrics['share_delta'] * 10000)

# Rank targets (largest score = biggest underperformance)
optim_targets = (
    leaf_metrics
    .sort_values('optimisation_score', ascending=False)
    .assign(
        total_cagr_pct=(root_cagr * 100).round(2),
        share_hist_pct=lambda d: (d['share_hist'] * 100).round(2),
        share_fcst_pct=lambda d: (d['share_fcst'] * 100).round(2),
    )
    .loc[:, ['Region', 'Method', 'Channel', 'cagr_pct', 'total_cagr_pct', 'growth_gap_pp', 'share_hist_pct', 'share_fcst_pct', 'share_delta_bp', 'optimisation_score', 'risk_factor']]
    .head(15)
)

print(f"Total CAGR over horizon: {root_cagr*100:.2f}%")
print("Top optimisation targets (leaf level):")
display(optim_targets)

# Visual: Top 10 channels at risk of market share loss (bps)
chart_targets = (
    leaf_metrics
    .sort_values('share_delta')
    .head(10)
    .copy()
)

chart_targets['label'] = chart_targets['Region'] + ' | ' + chart_targets['Channel']
chart_targets['label'] = (
    chart_targets['label']
    .str.replace('AllLoc__', '', regex=False)
    .str.replace('Domestic__', '', regex=False)
    .str.replace('_', ' ', regex=False)
)
chart_targets['share_delta_bps'] = chart_targets['share_delta'] * 10000

fig_targets = px.bar(
    chart_targets,
    x='share_delta_bps',
    y='label',
    orientation='h',
    title='Top 10 Channels at Risk of Market Share Loss',
    color='risk_factor',
    color_discrete_sequence=px.colors.qualitative.Prism,
    hover_data={'risk_factor': True, 'growth_gap_pp': ':.2f'},
)
fig_targets.update_layout(
    xaxis_title='Projected Market Share Change (bps)',
    yaxis_title='',
    yaxis=dict(categoryorder='array', categoryarray=list(chart_targets['label'])[::-1]),
    legend=dict(orientation='h', yanchor='top', y=-0.2, xanchor='center', x=0.5),
)
fig_targets.show()

Total CAGR over horizon: 5.47%
Top optimisation targets (leaf level):


,Region,Method,Channel,cagr_pct,total_cagr_pct,growth_gap_pp,share_hist_pct,share_fcst_pct,share_delta_bp,optimisation_score,risk_factor
34,North America,Card_eMoney,AllLoc__Magstripe,-46.38,5.47,51.85,0.52,0.07,-44.8,96.623315,Legacy Tech Sunset
19,Europe,Card_eMoney,AllLoc__Contactless,-2.12,5.47,7.59,1.41,1.13,-28.3,35.890870,Volume Contraction
21,Europe,Card_eMoney,Domestic__AllPresent,4.04,5.47,1.43,7.86,7.54,-31.5,32.971285,Competitive Erosion
27,LatAm,Card_eMoney,Domestic__AllPresent,2.81,5.47,2.66,3.84,3.55,-28.3,30.983049,Competitive Erosion
24,LatAm,Card_eMoney,AllLoc__AllPresent,2.91,5.47,2.56,3.63,3.37,-25.8,28.337910,Competitive Erosion
32,North America,Card_eMoney,AllLoc__AllPresent,4.83,5.47,0.64,14.63,14.37,-26.6,27.216135,Competitive Erosion
23,Europe,Card_eMoney,Domestic__Contactless,-10.10,5.47,15.57,0.06,0.04,-2.3,17.891120,Volume Contraction
33,North America,Card_eMoney,AllLoc__CardPresent,4.11,5.47,1.37,4.08,3.92,-15.7,17.017024,
29,MEA,Card_eMoney,AllLoc__AllPresent,1.99,5.47,3.48,0.96,0.87,-9.2,12.710451,
10,EMEA,Card_eMoney,AllLoc__CardPresent,1.52,5.47,3.95,0.48,0.43,-5.2,9.198415,


---
## Key Takeaways & Next Steps

### What This Analysis Reveals:
**Hierarchical forecasting** ensures coherent projections across all organisational levels  
**Risk segmentation** automatically flags channels requiring strategic intervention  
**Data-driven prioritisation** focuses resources on highest-impact optimisation opportunities

### Potential Applications:
1. **Merchant Strategy**: Target investment in declining channels (e.g., upgrade magstripe terminals)
2. **Product Roadmap**: Accelerate contactless/digital wallet adoption in underperforming regions
3. **Competitive Intelligence**: Monitor share shifts to detect emerging threats
4. **Executive Reporting**: Provide reconciled forecasts that stakeholders can trust

### Model Extensions:
- **Exogenous variables**: Add macroeconomic indicators (GDP, consumer spending) as regressors
- **Alternative reconcilers**: Test bottom-up, top-down, or middle-out reconciliation vs. MinTrace
- **Confidence intervals**: Surface forecast uncertainty for risk management
- **Automated alerts**: Flag channels when optimisation score exceeds threshold

---

**Data Source**: [BIS Committee on Payments and Market Infrastructures (CPMI)](https://www.bis.org/statistics/payment_stats.htm)  